# Regression with Taxi Dataset

This notebook demonstrates how to:

1. Define the model input and output schema
1. Load in data from a text file to an IDataView
1. Set up the training pipeline with data transforms
1. Choose an algorithm and append it to the pipeline
1. Train the model
1. Evaluate the model
1. Consume the model

## Install the necessary NuGet packages for training ML.NET model and plotting:

In [14]:
// using nightly-build
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-libraries/nuget/v3/index.json"
#r "nuget: Microsoft.ML.AutoML, 0.21.0-preview.23504.1"
#r "nuget: Microsoft.Data.Analysis, 0.21.0-preview.23504.1"
#r "nuget: Plotly.NET.Interactive, 3.0.2"
#r "nuget: Plotly.NET.CSharp, 0.0.1"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-libraries/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.21.0-preview.23504.1 Microsoft.ML.AutoML, 0.21.0-preview.23504.1 Plotly.NET.CSharp, 0.0.1 Plotly.NET.Interactive, 3.0.2

In [15]:

// Import common usings.
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;
using static Microsoft.ML.Transforms.OneHotEncodingEstimator;
using Microsoft.Data.Analysis;
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.AutoML;
using Microsoft.ML.Data;

#### Download or Locate Data
The following code tries to locate the data file in a few known locations or it will download it from the known GitHub location.

In [16]:
using System;
using System.IO;
using System.Net;

string EnsureDataSetDownloaded(string fileName)
{

	// This is the path if the repo has been checked out.
	var filePath = Path.Combine(Directory.GetCurrentDirectory(),"data", fileName);

	if (!File.Exists(filePath))
	{
		// This is the path if the file has already been downloaded.
		filePath = Path.Combine(Directory.GetCurrentDirectory(), fileName);
	}

	if (!File.Exists(filePath))
	{
		using (var client = new WebClient())
		{
			client.DownloadFile($"https://raw.githubusercontent.com/dotnet/csharp-notebooks/main/machine-learning/data/{fileName}", filePath);
		}
		Console.WriteLine($"Downloaded {fileName}  to : {filePath}");
	}
	else
	{
		Console.WriteLine($"{fileName} found here: {filePath}");
	}

	return filePath;
}

In [17]:
//Load File
var trainDataPath = EnsureDataSetDownloaded("taxi-fare.csv");
var df = DataFrame.LoadCsv(trainDataPath);
var mlContext = new MLContext();

// Append the trainer to the data processing pipeline
var pipeline = mlContext.Auto().Featurizer(df, excludeColumns: new[]{"fare_amount"})
                 .Append(mlContext.Auto().Regression(labelColumnName: "fare_amount"));

// Configure AutoML
var trainTestSplit = mlContext.Data.TrainTestSplit(df, 0.2);
var validateTestSplit = mlContext.Data.TrainTestSplit(trainTestSplit.TestSet, 0.5);
var monitor = new NotebookMonitor(pipeline);

 var experiment = mlContext.Auto().CreateExperiment()
                    .SetPipeline(pipeline)
                    .SetTrainingTimeInSeconds(50)
                    .SetDataset(trainTestSplit.TrainSet, validateTestSplit.TrainSet)
                    .SetRegressionMetric(RegressionMetric.RSquared, "fare_amount", "Score")
					.SetMonitor(monitor);

					// Configure Visualizer			
monitor.SetUpdate(monitor.Display());

// Start Experiment
var result = await experiment.RunAsync();

taxi-fare.csv found here: c:\Users\xiaoyuz\source\repos\csharp-notebooks\machine-learning\data\taxi-fare.csv


index,Trial,Metric,Trainer,Parameters
0,0,0.9191953,ReplaceMissingValues=>OneHotEncoding=>Concatenate=>FastForestRegression,"{""_pipeline_"":{""_SCHEMA_"":""e0 * e1 * e3 * e5"",""e0"":{""OutputColumnNames"":[""rate_code"",""passenger_count"",""trip_time_in_secs"",""trip_distance""],""InputColumnNames"":[""rate_code"",""passenger_count"",""trip_time_in_secs"",""trip_distance""]},""e1"":{""OutputColumnNames"":[""vendor_id"",""payment_type""],""InputColumnNames"":[""vendor_id"",""payment_type""]},""e3"":{""InputColumnNames"":[""rate_code"",""passenger_count"",""trip_time_in_secs"",""trip_distance"",""vendor_id"",""payment_type""],""OutputColumnName"":""Features""},""e5"":{""NumberOfTrees"":4,""NumberOfLeaves"":4,""FeatureFraction"":1,""LabelColumnName"":""fare_amount"",""FeatureColumnName"":""Features""}},""_SCHEMA_"":""e0 * e1 * e3 * e4"",""e0"":{""OutputColumnNames"":[""rate_code"",""passenger_count"",""trip_time_in_secs"",""trip_distance""],""InputColumnNames"":[""rate_code"",""passenger_count"",""trip_time_in_secs"",""trip_distance""]},""e1"":{""OutputColumnNames"":[""vendor_id"",""payment_type""],""InputColumnNames"":[""vendor_id"",""payment_type""]},""e2"":{""OutputColumnNames"":[""vendor_id"",""payment_type""],""InputColumnNames"":[""vendor_id"",""payment_type""]},""e3"":{""InputColumnNames"":[""rate_code"",""passenger_count"",""trip_time_in_secs"",""trip_distance"",""vendor_id"",""payment_type""],""OutputColumnName"":""Features""},""e4"":{""NumberOfLeaves"":4,""MinimumExampleCountPerLeaf"":20,""NumberOfTrees"":4,""MaximumBinCountPerFeature"":255,""FeatureFraction"":1,""LearningRate"":0.09999999999999998,""LabelColumnName"":""fare_amount"",""FeatureColumnName"":""Features"",""DiskTranspose"":false},""e5"":{""NumberOfTrees"":4,""NumberOfLeaves"":4,""FeatureFraction"":1,""LabelColumnName"":""fare_amount"",""FeatureColumnName"":""Features""},""e6"":{""NumberOfLeaves"":4,""MinimumExampleCountPerLeaf"":20,""LearningRate"":1,""NumberOfTrees"":4,""SubsampleFraction"":1,""MaximumBinCountPerFeature"":255,""FeatureFraction"":1,""L1Regularization"":2E-10,""L2Regularization"":1,""LabelColumnName"":""fare_amount"",""FeatureColumnName"":""Features""},""e7"":{""L1Regularization"":1,""L2Regularization"":1,""LabelColumnName"":""fare_amount"",""FeatureColumnName"":""Features""},""e8"":{""L1Regularization"":1,""L2Regularization"":0.1,""LabelColumnName"":""fare_amount"",""FeatureColumnName"":""Features""}}"
1,1,0.6837143,ReplaceMissingValues=>OneHotEncoding=>Concatenate=>SdcaRegression,"{""_pipeline_"":{""_SCHEMA_"":""e0 * e1 * e3 * e8"",""e0"":{""OutputColumnNames"":[""rate_code"",""passenger_count"",""trip_time_in_secs"",""trip_distance""],""InputColumnNames"":[""rate_code"",""passenger_count"",""trip_time_in_secs"",""trip_distance""]},""e1"":{""OutputColumnNames"":[""vendor_id"",""payment_type""],""InputColumnNames"":[""vendor_id"",""payment_type""]},""e3"":{""InputColumnNames"":[""rate_code"",""passenger_count"",""trip_time_in_secs"",""trip_distance"",""vendor_id"",""payment_type""],""OutputColumnName"":""Features""},""e8"":{""L1Regularization"":1,""L2Regularization"":0.1,""LabelColumnName"":""fare_amount"",""FeatureColumnName"":""Features""}},""_SCHEMA_"":""e0 * e1 * e3 * e4"",""e0"":{""OutputColumnNames"":[""rate_code"",""passenger_count"",""trip_time_in_secs"",""trip_distance""],""InputColumnNames"":[""rate_code"",""passenger_count"",""trip_time_in_secs"",""trip_distance""]},""e1"":{""OutputColumnNames"":[""vendor_id"",""payment_type""],""InputColumnNames"":[""vendor_id"",""payment_type""]},""e2"":{""OutputColumnNames"":[""vendor_id"",""payment_type""],""InputColumnNames"":[""vendor_id"",""payment_type""]},""e3"":{""InputColumnNames"":[""rate_code"",""passenger_count"",""trip_time_in_secs"",""trip_distance"",""vendor_id"",""payment_type""],""OutputColumnName"":""Features""},""e4"":{""NumberOfLeaves"":4,""MinimumExampleCountPerLeaf"":20,""NumberOfTrees"":4,""MaximumBinCountPerFeature"":255,""FeatureFraction"":1,""LearningRate"":0.099999

## Consume the model

In [18]:
// Define sample data
var data  = new DataFrame(new StringDataFrameColumn("vendor_id"), new PrimitiveDataFrameColumn<float>("rate_code"), new PrimitiveDataFrameColumn<float>("passenger_count"), new PrimitiveDataFrameColumn<float>("trip_time_in_secs"), new PrimitiveDataFrameColumn<float>("trip_distance"),new StringDataFrameColumn("payment_type"));
data.Append(new List<KeyValuePair<string,object>>()
{
	new KeyValuePair<string,object>("vendor_id",@"CMT"),
	new KeyValuePair<string,object>("rate_code",1F),
	new KeyValuePair<string,object>("passenger_count",1F),
	new KeyValuePair<string, object> ("trip_time_in_secs",474F),
	new KeyValuePair<string, object> ("trip_distance",1.5F),
	new KeyValuePair<string, object> ("payment_type",@"payment_type")
},true);
var model = result.Model;

//Use the model to transform the sample data.
var output = model.Transform(data);

// Get the predicted score with the sample data.
var predictedScore = output.GetColumn<float>("Score");
predictedScore
			
			

[ 8.5999565 ]

## Evaluate model

In [19]:
var model = result.Model;
var eval= model.Transform(validateTestSplit.TestSet);
var metric=mlContext.Regression.Evaluate(eval,"fare_amount");
metric	

MeanAbsoluteError,0.8986130748998551
MeanSquaredError,7.677836035532971
RootMeanSquaredError,2.770890837895454
LossFunction,7.677836067929183
RSquared,0.9168392253985385




The code below demonstrates several methods to explain your model, including how to get and display

1. A Histogram of the distribution of number of instances
1. A Scatter Plot
1. Compare actual values to predicted values in a scatter plot
1. The importance of different features

In [20]:
using System;
using System.Collections.Generic;
using System.Collections.Immutable;
using System.IO;
using System.Linq;
using Plotly.NET.CSharp;
using Plotly.NET;

## Compare Distribution of Number of Instances

In [21]:
// Extract some data into arrays for plotting

int numberOfRows = 1000;

// Columns was determined by inputted data
float[] fare_amount = df.GetColumn<float>("fare_amount").Take(numberOfRows).ToArray();

// Distribution of Number of Instances

Plotly.NET.Chart2D.Chart.Histogram<float,float, string>(fare_amount)
.WithXAxisStyle<float, float, string>(TitleText: "fare_amount")
.WithYAxisStyle<float, float, string>(TitleText: "Number of Instances")

<!-- Plotly chart will be drawn inside this DIV -->

## Compare actual values to predicted values in a scatter plot

In [22]:
// Number of rows to display in charts.
int numberOfRows = 1000;
// Use the model to make batch predictions on training data
var testResults = model.Transform(df);

// Get the actual values from the dataset
var trueValues = testResults.GetColumn<float>("fare_amount").Take(numberOfRows);;

// Get the predicted values from the test results
var predictedValues = testResults.GetColumn<float>("Score").Take(numberOfRows);

var scatter = Plotly.NET.CSharp.Chart.Scatter<float, float, string>(x: trueValues, y: predictedValues, mode: Plotly.NET.StyleParam.Mode.Markers)
.WithTraceInfo("True Values, Predicted Values", ShowLegend: true);

var maximumValue = Math.Max(trueValues.Max(), predictedValues.Max());
var perfectLine = Plotly.NET.CSharp.Chart.Scatter<float, float, string>(x:new[] {0, maximumValue}, y: new[] {0, maximumValue}, mode: Plotly.NET.StyleParam.Mode.Lines)
.WithTraceInfo("Perfect Line", ShowLegend: true);


Plotly.NET.CSharp.Chart.Combine(new []{scatter, perfectLine})
.WithXAxisStyle<double, double, string>(TitleText: "X", ShowGrid: false)

<!-- Plotly chart will be drawn inside this DIV -->

## Calculate and graph the Permutation Feature Importance (PFI)

In [23]:
// Calculate PFI
var preprocessedTrainData = model.Transform(df);

ImmutableDictionary<string, RegressionMetricsStatistics> permutationFeatureImportance =
    mlContext.Regression
	.PermutationFeatureImportance(
                model,
                preprocessedTrainData,
                labelColumnName: "fare_amount",
                useFeatureWeightFilter: false,
                numberOfExamplesToUse: null,
                permutationCount: 1);

var featureImportanceMetrics =
     permutationFeatureImportance
     .Select((kvp) => new { kvp.Key, kvp.Value.RSquared })
     .OrderByDescending(myFeatures => Math.Abs(myFeatures.RSquared.Mean));

    
var featureNames = new List<string>();
var featurePFI = new List<double>();
foreach (var feature in featureImportanceMetrics)
{
     featureNames.Add(feature.Key);
     featurePFI.Add(Math.Abs(feature.RSquared.Mean));
}
var featureImportance = new DataFrame(new StringDataFrameColumn("Feature", featureNames.ToArray() ), new DoubleDataFrameColumn("R-Squared Impact",featurePFI.ToArray()));
    
featureImportance

index,Feature,R-Squared Impact
0,trip_distance,0.69333761241898
1,rate_code,0.2915977337251443
2,trip_time_in_secs,0.15290071900531021
3,payment_type.UNK,0
4,passenger_count,0
5,payment_type.DIS,0
6,payment_type.NOC,0
7,vendor_id.CMT,0
8,payment_type.CRD,0
9,vendor_id.VTS,0


In [ ]:
Plotly.NET.CSharp.Chart.Bar<double, string, string>(values: featurePFI, Keys: featureNames)
.WithTraceInfo(Name: "Hello from C#", ShowLegend: false)
.WithXAxisStyle<double, double, string>(TitleText: "Feature")
.WithYAxisStyle<double, double, string>(TitleText: "Contribution (delta R-Squared)")

<!-- Plotly chart will be drawn inside this DIV -->

In [13]:
var topFeatureName = featureNames.First();
float[] fare_amount = df.GetColumn<float>("fare_amount").Take(numberOfRows).ToArray();
float[] topFeature = df.GetColumn<float>(topFeatureName).Take(numberOfRows).ToArray();

Plotly.NET.CSharp.Chart.Point<float, float, string>(topFeature, fare_amount, "Hello")
.WithTraceInfo(Name: "Hello from C#", ShowLegend: true)
.WithXAxisStyle<float, float, string>(TitleText: topFeatureName, ShowGrid: false)
.WithYAxisStyle<float, float, string>(TitleText: "fare_amount", ShowGrid: false)

Error: (1,22): error CS0103: The name 'featureNames' does not exist in the current context
(2,23): error CS0103: The name 'df' does not exist in the current context
(2,63): error CS0103: The name 'numberOfRows' does not exist in the current context
(3,22): error CS0103: The name 'df' does not exist in the current context
(3,63): error CS0103: The name 'numberOfRows' does not exist in the current context